In [161]:
import pandas as pd
import numpy as np
import seaborn as sb

In [162]:
v = pd.read_csv("../../data/processed/V.csv")
fgr = pd.read_csv("../../data/processed/FGR.csv", decimal=",")
paradas = pd.read_csv("../../data/raw/paradas.csv")
rota_paradas = pd.read_csv("../../data/raw/rota_paradas.csv")
rotas = pd.read_csv("../../data/raw/rotas.csv")
dados_bairros = pd.read_csv("../../data/raw/dados_bairros.csv")

- Demanda
- Tamanho da linha
- Horario
- IDH dos bairros
- É domingo?

In [163]:
demanda_p_linha_dia = v.groupby(["linha_numero", "movimento_diario_data_mov"]).count()["passageiro_matricula"].to_frame().reset_index()
demanda_p_linha_dia.rename(columns={"passageiro_matricula": "demanda_do_dia"}, inplace = True)

In [164]:
demanda_p_linha_dia

,linha_numero,movimento_diario_data_mov,demanda_do_dia
0,1,2021-08-29,232
1,1,2021-08-31,9096
2,1,2021-09-01,8888
3,1,2021-09-02,8244
4,1,2021-09-03,9020
...,...,...,...
2078,942,2021-09-05,2
2079,991,2021-08-31,76
2080,991,2021-09-01,55
2081,991,2021-09-02,55


In [165]:
# Demanda por intervalor de tempo
v["passageiro_data_hora"] = pd.to_datetime(v["passageiro_data_hora"])
v["passageiro_hora_entrada"] = v["passageiro_data_hora"].dt.hour
v["passageiro_hora_range"] = pd.cut(v["passageiro_hora_entrada"], bins=[-1, 4, 6, 9, 12, 15, 18, 21, float('Inf')], labels=['00-3h59', '4-5h59', '6-8h59', '9-11h59', '12-14h59', '15-17h59', '18-20h59', '21-23h59'])
v_hora_range = v.groupby(["linha_numero", "movimento_diario_data_mov", "passageiro_hora_range"]).count()["passageiro_matricula"].to_frame().reset_index()
v_hora_range.dropna(inplace=True)

In [166]:
v_hora_range

,linha_numero,movimento_diario_data_mov,passageiro_hora_range,passageiro_matricula
73,1,2021-08-29,4-5h59,4.0
74,1,2021-08-29,6-8h59,74.0
75,1,2021-08-29,9-11h59,97.0
76,1,2021-08-29,12-14h59,57.0
88,1,2021-08-31,00-3h59,150.0
...,...,...,...,...
47973,991,2021-08-31,15-17h59,75.0
47981,991,2021-09-01,15-17h59,55.0
47989,991,2021-09-02,15-17h59,55.0
48164,999,2021-09-05,12-14h59,82.0


In [167]:
v_hora_range.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14495 entries, 73 to 48165
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   linha_numero               14495 non-null  int64   
 1   movimento_diario_data_mov  14495 non-null  object  
 2   passageiro_hora_range      14495 non-null  category
 3   passageiro_matricula       14495 non-null  float64 
dtypes: category(1), float64(1), int64(1), object(1)
memory usage: 467.5+ KB


In [168]:
days_week ={
    0: 'Segunda', 
    1: 'Terça', 
    2: 'Quarta', 
    3: 'Quinta', 
    4: 'Sexta',
    5: 'Sábado', 
    6: 'Domingo'
}

v_hora_range["movimento_diario_data_mov"] = pd.to_datetime(v_hora_range["movimento_diario_data_mov"])

def eh_domingo(dia_semana):
    if dia_semana == "Domingo":
        return 1
    else:
        return 0

v_hora_range["Dia_Semana"] = v_hora_range.movimento_diario_data_mov.dt.weekday
v_hora_range["Dia_Semana"] = v_hora_range.Dia_Semana.map(days_week)
v_hora_range["Domingo"] = v_hora_range["Dia_Semana"].map(eh_domingo)

In [169]:
v_hora_range

,linha_numero,movimento_diario_data_mov,passageiro_hora_range,passageiro_matricula,Dia_Semana,Domingo
73,1,2021-08-29,4-5h59,4.0,Domingo,1
74,1,2021-08-29,6-8h59,74.0,Domingo,1
75,1,2021-08-29,9-11h59,97.0,Domingo,1
76,1,2021-08-29,12-14h59,57.0,Domingo,1
88,1,2021-08-31,00-3h59,150.0,Terça,0
...,...,...,...,...,...,...
47973,991,2021-08-31,15-17h59,75.0,Terça,0
47981,991,2021-09-01,15-17h59,55.0,Quarta,0
47989,991,2021-09-02,15-17h59,55.0,Quinta,0
48164,999,2021-09-05,12-14h59,82.0,Domingo,1


In [170]:
# Variável de km da rota
linha_p_km_programado = fgr[["fechamento_data", "linha_numero", "linha_km_programado"]].drop_duplicates()
linha_p_km_programado = linha_p_km_programado[linha_p_km_programado.linha_km_programado != 0]
linha_demanda_km = demanda_p_linha_dia.merge(linha_p_km_programado, how="inner", left_on=["linha_numero", "movimento_diario_data_mov"], right_on=["linha_numero","fechamento_data"])
linha_demanda_km.drop(columns=["fechamento_data"], inplace=True)
linha_demanda_km = linha_demanda_km.groupby(["linha_numero"]).mean()[["linha_km_programado"]].reset_index()
dataset_model = v_hora_range.merge(linha_demanda_km, how="inner", on="linha_numero")

In [171]:
rota_paradas_merged = rota_paradas.merge(paradas, how="inner", on="stop_id")
rota_paradas_merged = rota_paradas_merged.merge(rotas, how="inner", on="route_id")
rotas_paradas_clean = rota_paradas_merged[["route_id", "stop_sequence", "district", "city"]]
rotas_paradas_demanda = rotas_paradas_clean.merge(demanda_p_linha_dia, how="inner", left_on="route_id", right_on="linha_numero")
rotas_paradas_demanda.drop(columns=["route_id", "stop_sequence"], inplace=True)
rotas_paradas_demanda = rotas_paradas_demanda[rotas_paradas_demanda["city"] == "Fortaleza"]

In [172]:
# Retira acentos e barras (/ + termo) dos nomes dos bairros
from unidecode import unidecode

district = []
for index, row in rotas_paradas_demanda.iterrows():
  district_without_punt = unidecode(row["district"])
  district_without_barra = district_without_punt.split("/")[0]
  if district_without_barra.startswith("Parque "):
    district_without_barra = district_without_barra.replace("Parque ", "")
  if district_without_barra.startswith("Vila "):
    district_without_barra = district_without_barra.replace("Vila ", "")
  if district_without_barra.startswith("Prefeito "):
    district_without_barra = district_without_barra.replace("Prefeito ", "")
  district.append(district_without_barra.strip())
rotas_paradas_demanda["Bairro"] = district

district = []
for index, row in dados_bairros.iterrows():
  district_without_punt = unidecode(row["Bairros"])
  district_without_barra = district_without_punt.split("/")[0]
  if district_without_barra.startswith("Parque "):
    district_without_barra = district_without_barra.replace("Parque ", "")
  if district_without_barra.startswith("Vila "):
    district_without_barra = district_without_barra.replace("Vila ", "")
  if district_without_barra.startswith("Prefeito "):
    district_without_barra = district_without_barra.replace("Prefeito ", "")
  district.append(district_without_barra.strip())
dados_bairros["Bairro"] = district

In [173]:
rotas_paradas_demanda["Bairro"] = rotas_paradas_demanda["Bairro"].str.lower()
rotas_paradas_demanda["Bairro"] = rotas_paradas_demanda["Bairro"].str.replace("ç", "c")
dados_bairros["Bairro"] = dados_bairros["Bairro"].str.lower()

In [174]:
rotas_demanda_bairro = rotas_paradas_demanda.merge(dados_bairros, how="inner", left_on="Bairro", right_on="Bairro")
rotas_media_idh = rotas_demanda_bairro.groupby(["linha_numero"]).mean()[["IDH em 2010[8]"]]

In [175]:
dataset_model = dataset_model.merge(rotas_media_idh, how="inner", on="linha_numero")

In [176]:
rotas_idh_max = rotas_demanda_bairro.groupby(["linha_numero"]).max()[["IDH em 2010[8]"]].reset_index()

In [177]:
dataset_model = dataset_model.merge(rotas_idh_max, how="inner", on="linha_numero")

GET DUMMIES HORA RANGE

In [180]:
dataset_model = pd.concat([dataset_model, pd.get_dummies(dataset_model["passageiro_hora_range"])], axis = 1)

In [181]:
dataset_model.drop(columns=["Dia_Semana", "movimento_diario_data_mov", "linha_numero", "passageiro_hora_range"], inplace=True)

In [183]:
dataset_model

,passageiro_matricula,Domingo,linha_km_programado,IDH em 2010[8]_x,IDH em 2010[8]_y,00-3h59,4-5h59,6-8h59,9-11h59,12-14h59,15-17h59,18-20h59,21-23h59
0,319.0,0,1165.120,0.465660,0.762,0,0,1,0,0,0,0,0
1,398.0,0,1165.120,0.465660,0.762,0,0,0,1,0,0,0,0
2,557.0,0,1165.120,0.465660,0.762,0,0,0,0,1,0,0,0
3,979.0,0,1165.120,0.465660,0.762,0,0,0,0,0,1,0,0
4,76.0,0,1165.120,0.465660,0.762,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12465,282.0,0,1157.175,0.291214,0.530,0,0,0,1,0,0,0,0
12466,352.0,0,1157.175,0.291214,0.530,0,0,0,0,1,0,0,0
12467,614.0,0,1157.175,0.291214,0.530,0,0,0,0,0,1,0,0
12468,185.0,0,1157.175,0.291214,0.530,0,0,0,0,0,0,1,0


In [184]:
dataset_model.rename(columns={"passageiro_hora_range": "hora_range", 
                              "passageiro_matricula": "demanda",
                              "Domingo": "domingo", 
                              "linha_km_programado": "linha_km",
                              "IDH em 2010[8]_x": "media_idh",
                              "IDH em 2010[8]_y": "idh_max"}, inplace=True)

In [185]:
dataset_model

,demanda,domingo,linha_km,media_idh,idh_max,00-3h59,4-5h59,6-8h59,9-11h59,12-14h59,15-17h59,18-20h59,21-23h59
0,319.0,0,1165.120,0.465660,0.762,0,0,1,0,0,0,0,0
1,398.0,0,1165.120,0.465660,0.762,0,0,0,1,0,0,0,0
2,557.0,0,1165.120,0.465660,0.762,0,0,0,0,1,0,0,0
3,979.0,0,1165.120,0.465660,0.762,0,0,0,0,0,1,0,0
4,76.0,0,1165.120,0.465660,0.762,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12465,282.0,0,1157.175,0.291214,0.530,0,0,0,1,0,0,0,0
12466,352.0,0,1157.175,0.291214,0.530,0,0,0,0,1,0,0,0
12467,614.0,0,1157.175,0.291214,0.530,0,0,0,0,0,1,0,0
12468,185.0,0,1157.175,0.291214,0.530,0,0,0,0,0,0,1,0


# Pre Processamento

In [186]:
X = dataset_model.drop(columns=["demanda"]).values
Y = dataset_model["demanda"].values

In [187]:
from sklearn.preprocessing import StandardScaler
standard = StandardScaler()

X = standard.fit_transform(X)

In [188]:
X

array([[-0.37452726,  0.50449666,  0.78716743, ..., -0.39214153,
        -0.39173309, -0.36174662],
       [-0.37452726,  0.50449666,  0.78716743, ..., -0.39214153,
        -0.39173309, -0.36174662],
       [-0.37452726,  0.50449666,  0.78716743, ..., -0.39214153,
        -0.39173309, -0.36174662],
       ...,
       [-0.37452726,  0.49316187, -0.59505574, ...,  2.55009969,
        -0.39173309, -0.36174662],
       [-0.37452726,  0.49316187, -0.59505574, ..., -0.39214153,
         2.55275856, -0.36174662],
       [-0.37452726,  0.49316187, -0.59505574, ..., -0.39214153,
        -0.39173309,  2.76436587]])

In [228]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import ElasticNet 
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [229]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.75, test_size=0.25, random_state=123)

In [230]:
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)
results = linear_regression.predict(X_test)

In [231]:
results

array([ 227.20173994, -195.92326006,  -61.92326006, ...,  -26.29826006,
        160.07673994,  383.07673994])

In [232]:
mean_absolute_error(y_test, results)

146.32499474400072

In [233]:
np.sqrt(mean_squared_error(y_test, results))

223.6511472982105

In [234]:
r2_score(y_test, results)

0.49062270740133473

Lasso

In [235]:
lasso = Lasso()
lasso.fit(X_train, y_train)
results = lasso.predict(X_test)

In [236]:
mean_absolute_error(y_test, results)

145.76145959707588

In [237]:
np.sqrt(mean_squared_error(y_test, results))

223.57570468197818

In [238]:
r2_score(y_test, results)

0.49096629852188745

Ridge

In [239]:
ridge_classifier = RidgeClassifier()
ridge_classifier.fit(X_train, y_train)
results = ridge_classifier.predict(X_test)

In [240]:
mean_absolute_error(y_test, results)

177.89127645926877

In [241]:
np.sqrt(mean_squared_error(y_test, results))

332.8118059864717

In [242]:
r2_score(y_test, results)

-0.12796277583758542

ElasticNet

In [243]:
elastic_net = ElasticNet()
elastic_net.fit(X_train, y_train)
results = elastic_net.predict(X_test)

In [244]:
mean_absolute_error(y_test, results)

144.03849382971123

In [245]:
np.sqrt(mean_squared_error(y_test, results))

233.9209566871839

In [246]:
r2_score(y_test, results)

0.4427685974927017